# Managing State

So far, we have discussed how to build components in a multi-agent application - agents, teams, termination conditions. In many cases, it is useful to save the state of these components to disk and load them back later. This is particularly useful in a web application where stateless endpoints respond to requests and need to load the state of the application from persistent storage.

In this notebook, we will discuss how to save and load the state of agents, teams, and termination conditions.

## Saving and Loading Agents

We can get the state of an agent by calling `save_state()` method on an [AssistantAgent](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.AssistantAgent).

**⚠️ Warning**: *For [AssistantAgent](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.AssistantAgent), its state consists of the model_context. If you write your own custom agent, consider overriding the [save_state()](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.BaseChatAgent.save_state) and [load_state()](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.BaseChatAgent.load_state) methods to customize the behavior. The default implementations save and load an empty state.*

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(model="gpt-4o-2024-08-06")

assistant_agent = AssistantAgent(
    name="assistant_agent",
    system_message="You are a helpful assistant",
    model_client=model_client,
)

# Use asyncio.run(...) when running in a script.
response = await assistant_agent.on_messages(
    [TextMessage(content="Write a 3 line poem on lake tangayika", source="user")], CancellationToken()
)
print(response.chat_message)
await model_client.close()

id='b7596cd5-0f73-4c0d-8513-df17e5dd82df' source='assistant_agent' models_usage=RequestUsage(prompt_tokens=28, completion_tokens=29) metadata={} created_at=datetime.datetime(2025, 7, 28, 6, 54, 19, 150311, tzinfo=datetime.timezone.utc) content="In Tanganyika's depths so wide and blue,  \nWhispers of ancient stories softly renew,  \nMirror to sunsets, a tranquil view.  " type='TextMessage'


In [2]:
agent_state = await assistant_agent.save_state()
print(agent_state)

{'type': 'AssistantAgentState', 'version': '1.0.0', 'llm_context': {'messages': [{'content': 'Write a 3 line poem on lake tangayika', 'source': 'user', 'type': 'UserMessage'}, {'content': "In Tanganyika's depths so wide and blue,  \nWhispers of ancient stories softly renew,  \nMirror to sunsets, a tranquil view.  ", 'thought': None, 'source': 'assistant_agent', 'type': 'AssistantMessage'}]}}


In [3]:
model_client = OpenAIChatCompletionClient(model="gpt-4o-2024-08-06")

new_assistant_agent = AssistantAgent(
    name="assistant_agent",
    system_message="You are a helpful assistant",
    model_client=model_client,
)
await new_assistant_agent.load_state(agent_state)

# Use asyncio.run(...) when running in a script.
response = await new_assistant_agent.on_messages(
    [TextMessage(content="What was the last line of the previous poem you wrote", source="user")], CancellationToken()
)
print(response.chat_message)
await model_client.close()

id='7a5e736a-8044-4186-8300-99f0cc183a0d' source='assistant_agent' models_usage=RequestUsage(prompt_tokens=77, completion_tokens=22) metadata={} created_at=datetime.datetime(2025, 7, 28, 6, 55, 15, 223443, tzinfo=datetime.timezone.utc) content='The last line of the poem I wrote for you is:  \n"Mirror to sunsets, a tranquil view."' type='TextMessage'


## Saving and Loading Teams

We can get the state of a team by calling `save_state` method on the team and load it back by calling `load_state` method on the team.

When we call `save_state` on a team, it saves the state of all the agents in the team.

We will begin by creating a simple [RoundRobinGroupChat](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.RoundRobinGroupChat) team with a single agent and ask it to write a poem.

In [4]:
model_client = OpenAIChatCompletionClient(model="gpt-4o-2024-08-06")

# Define a team.
assistant_agent = AssistantAgent(
    name="assistant_agent",
    system_message="You are a helpful assistant",
    model_client=model_client,
)
agent_team = RoundRobinGroupChat([assistant_agent], termination_condition=MaxMessageTermination(max_messages=2))

# Run the team and stream messages to the console.
stream = agent_team.run_stream(task="Write a beautiful poem 3-line about lake tangayika")

# Use asyncio.run(...) when running in a script.
await Console(stream)

# Save the state of the agent team.
team_state = await agent_team.save_state()

---------- TextMessage (user) ----------
Write a beautiful poem 3-line about lake tangayika
---------- TextMessage (assistant_agent) ----------
In Tanganyika's depths, whispers of ancient tales,  
Crystal waves embrace shores with gentle ballet,  
Serene heart of Africa, where spirits sail.


If we reset the team (simulating instantiation of the team), and ask the question `What was the last line of the poem you wrote?`, we see that the team is unable to accomplish this as there is no reference to the previous run.

In [5]:
await agent_team.reset()

stream = agent_team.run_stream(task="What was the last line of the poem you wrote?")

await Console(stream)

---------- TextMessage (user) ----------
What was the last line of the poem you wrote?
---------- TextMessage (assistant_agent) ----------
I don't have the ability to recall previous interactions or specific writings since I don't retain user data or past conversations. However, I can certainly help you write a new poem or provide another line if you'd like. Let me know what you'd prefer!


TaskResult(messages=[TextMessage(id='0cfe308e-1a99-4cf2-a1ac-71b462fa6da3', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 28, 7, 1, 6, 276204, tzinfo=datetime.timezone.utc), content='What was the last line of the poem you wrote?', type='TextMessage'), TextMessage(id='34cdd8f6-0c85-48cb-bbbf-508939cb190e', source='assistant_agent', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=48), metadata={}, created_at=datetime.datetime(2025, 7, 28, 7, 1, 8, 72276, tzinfo=datetime.timezone.utc), content="I don't have the ability to recall previous interactions or specific writings since I don't retain user data or past conversations. However, I can certainly help you write a new poem or provide another line if you'd like. Let me know what you'd prefer!", type='TextMessage')], stop_reason='Maximum number of messages 2 reached, current message count: 2')

Next, we load the state of the team and ask the same question. We see that the team is able to accurately return the last line of the poem it wrote.

In [6]:
print(team_state)

# Load team state.
await agent_team.load_state(team_state)

stream = agent_team.run_stream(task="What was the last line of the poem you wrote?")

await Console(stream)

{'type': 'TeamState', 'version': '1.0.0', 'agent_states': {'assistant_agent': {'type': 'ChatAgentContainerState', 'version': '1.0.0', 'agent_state': {'type': 'AssistantAgentState', 'version': '1.0.0', 'llm_context': {'messages': [{'content': 'Write a beautiful poem 3-line about lake tangayika', 'source': 'user', 'type': 'UserMessage'}, {'content': "In Tanganyika's depths, whispers of ancient tales,  \nCrystal waves embrace shores with gentle ballet,  \nSerene heart of Africa, where spirits sail.", 'thought': None, 'source': 'assistant_agent', 'type': 'AssistantMessage'}]}}, 'message_buffer': []}, 'RoundRobinGroupChatManager': {'type': 'RoundRobinManagerState', 'version': '1.0.0', 'message_thread': [{'id': '8a3e5e78-8ca4-4aed-8540-655f2a1728f1', 'source': 'user', 'models_usage': None, 'metadata': {}, 'created_at': datetime.datetime(2025, 7, 28, 7, 0, 19, 97065, tzinfo=datetime.timezone.utc), 'content': 'Write a beautiful poem 3-line about lake tangayika', 'type': 'TextMessage'}, {'id': 

TaskResult(messages=[TextMessage(id='ef41f83c-db75-4252-93ab-e7f9992cb228', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 28, 7, 1, 28, 357854, tzinfo=datetime.timezone.utc), content='What was the last line of the poem you wrote?', type='TextMessage'), TextMessage(id='db7aed3f-f817-4005-9d87-ec412e9a2f6e', source='assistant_agent', models_usage=RequestUsage(prompt_tokens=80, completion_tokens=20), metadata={}, created_at=datetime.datetime(2025, 7, 28, 7, 1, 29, 639023, tzinfo=datetime.timezone.utc), content='The last line of the poem is:  \n"Serene heart of Africa, where spirits sail."', type='TextMessage')], stop_reason='Maximum number of messages 2 reached, current message count: 2')

## Persisting State (File or Database)

In [9]:
team_state

{'type': 'TeamState',
 'version': '1.0.0',
 'agent_states': {'assistant_agent': {'type': 'ChatAgentContainerState',
   'version': '1.0.0',
   'agent_state': {'type': 'AssistantAgentState',
    'version': '1.0.0',
    'llm_context': {'messages': [{'content': 'Write a beautiful poem 3-line about lake tangayika',
       'source': 'user',
       'type': 'UserMessage'},
      {'content': "In Tanganyika's depths, whispers of ancient tales,  \nCrystal waves embrace shores with gentle ballet,  \nSerene heart of Africa, where spirits sail.",
       'thought': None,
       'source': 'assistant_agent',
       'type': 'AssistantMessage'}]}},
   'message_buffer': []},
  'RoundRobinGroupChatManager': {'type': 'RoundRobinManagerState',
   'version': '1.0.0',
   'message_thread': [{'id': '8a3e5e78-8ca4-4aed-8540-655f2a1728f1',
     'source': 'user',
     'models_usage': None,
     'metadata': {},
     'created_at': datetime.datetime(2025, 7, 28, 7, 0, 19, 97065, tzinfo=datetime.timezone.utc),
     'co

In [ ]:
import json

## save state to disk

with open(".team_state.json", "w") as f:
    json.dump(team_state, f)

## load state from disk
with open(".team_state.json", "r") as f:
    team_state = json.load(f)

new_agent_team = RoundRobinGroupChat([assistant_agent], termination_condition=MaxMessageTermination(max_messages=2))

await new_agent_team.load_state(team_state)

stream = new_agent_team.run_stream(task="What was the last line of the poem you wrote?")

await Console(stream)

await model_client.close()